In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import rbf_kernel

In [2]:
def get_data(path):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    df = pd.read_csv(path)
    return(df)

In [3]:
def train_test_split(df, train_percent=0.666666):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    np.random.seed(4109)
    df = df.sample(frac=1).reset_index(drop=True)
    
    n_train = round(train_percent*len(df))
    df_train = df[:n_train]
    df_test = df[n_train:]
    
    return(df_train, df_test)

In [4]:
def get_grid_search_vals(start_g, stop_g, start_s, stop_s):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    gamma_powers = np.arange(start_g, stop_g + 1, 1)
    sigma_powers = np.arange(start_s, stop_s + 1, 0.5)
    
    gamma = np.power(0.5, gamma_powers)
    sigma = np.power(2, sigma_powers)
    
    return(gamma, sigma)

In [5]:
def get_pairwise_dist(A, B):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    
    References for einsum:
    https://ajcr.net/Basic-guide-to-einsum/
    https://stackoverflow.com/questions/42991347/how-to-find-the-pairwise-differences-between-rows-of-two-very-large-matrices-usi
    -------------------------
    '''
    # Element wise product and then sum horizontally
    # Element wise product and then sum horizontally
    # Dot product between A and B.T
    dist = np.einsum('ij,ij->i',A, A)[:,None] + np.einsum('ij,ij->i',B,B) - 2*np.dot(A,B.T)
    return(dist)

In [6]:
def test_pairwise_dist(A, B):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    
    References for einsum:
    https://ajcr.net/Basic-guide-to-einsum/
    https://stackoverflow.com/questions/45896939/using-python-numpy-einsum-to-obtain-dot-product-between-2-matrices
    -------------------------
    '''
    dist = get_pairwise_dist(A, B)
    dist_sk = cdist(A, B)**2
    return(dist, dist_sk, np.allclose(dist, dist_sk))

In [7]:
def get_gaussian_kernel(X, X_test, sigma):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    scale = -0.5/np.power(sigma, 2)
    K = get_pairwise_dist(X, X_test) 
    K = K*scale
    K = np.exp(K)
    return(K)

In [8]:
def test_gaussian_kernel(X, X_test, sigma):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    kernel = get_gaussian_kernel(X, X_test, sigma)
    kernel_sk = rbf_kernel(X, X_test, 0.5)
    return(np.allclose(kernel, kernel_sk))

In [9]:
def get_kernel_ridge_coeff(X, Y, sigma, gamma):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    m = Y.shape[0]
    K = get_gaussian_kernel(X, X, sigma)
    
    alpha = np.linalg.solve(K + gamma*np.identity(m), Y)
    alpha = alpha.reshape(alpha.shape[0], 1)
    
    return(alpha)

In [10]:
def test_kernel_ridge_coeff():
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    a = np.array([1, 2, 3, 4, 5, 6, 7, 8]).reshape(2,4)
    b = np.array([2, 4]).reshape(2,1)
    result = get_kernel_ridge_coeff(K, b, 1, 2)
    return(result)

In [11]:
def get_kernel_ridge_prediction(alpha, X, X_test, sigma):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    K_test = get_gaussian_kernel(X, X_test, sigma)
    # Multiply element wise and then sum downwards to get # test points columns
    y_pred = np.einsum('ij,ij->j', alpha, K_test)
    return(y_pred)

In [12]:
def test_kernel_ridge_prediction():
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    a = np.array([1, 2, 3, 4, 5, 6, 7, 8]).reshape(2,4)
    b = np.array([5, 8, 9, 10, 11, 12, 13, 14, 18, 19, 20, 21, 25, 26, 7, 8]).reshape(4,4)
    
    alpha = test_kernel_ridge_coeff()
    result = get_kernel_ridge_prediction(alpha, a, b, 1)
    
    return(result)

In [13]:
def get_k_folds(df_train, k = 5):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    # Set the seed to make sure reproducible
    # Then create a list of folds
    np.random.seed(43890)
    fold_size = round(len(df_train)/k)
    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_folds = [df_train.iloc[i:i+fold_size] for i in range(0,len(df_train)-fold_size+1,fold_size)]
    return(df_folds)

In [14]:
def get_mse(Y, Y_hat):
    '''
    ------------------------
    Input: True values and predicted values
    Output: Mean squared error
    ------------------------
    '''
    return np.sum(np.power(Y-Y_hat, 2))/max(Y.shape)

In [15]:
def plot_mse(gamma, sigma):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    pass

In [16]:
def train_kernel_ridge(X_train, X_val, Y_train, Y_val, sigma, gamma):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    # Get dual form estimates
    alpha = get_kernel_ridge_coeff(X_train, Y_train, sigma, gamma)

    # Get predictions
    y_pred_train = get_kernel_ridge_prediction(alpha, X_train, X_train, sigma)
    y_pred_val = get_kernel_ridge_prediction(alpha, X_train, X_val, sigma)
    
    # Get loss values
    mse_train = get_mse(Y_train, y_pred_train)
    mse_val = get_mse(Y_val, y_pred_val)
    
    return(alpha, y_pred_train, y_pred_val, mse_train, mse_val)

In [17]:
def get_best_params(history):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    best_run = np.array(history['mse_val']).argmin()
    best_val_mse = np.array(history['mse_val']).min()
    
    best_gamma, best_sigma = history['gamma'][best_run], history['sigma'][best_run]
    best_alpha = history['alpha'][best_run]
    
    best_params = {'best_run': best_run, 
                   'best_gamma': best_gamma, 
                   'best_sigma': best_sigma, 
                   'best_val_mse': best_val_mse, 
                   'best_run_alpha': best_alpha}
    
    return(best_params)

In [18]:
def tests():
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    # Make test data
    a = np.array([1, 2, 3, 4, 5, 6, 7, 8]).reshape(2,4)
    b = np.array([2, 4, 6, 8, 10, 12, 14, 16]).reshape(2,4)
    
    dist_test_result = test_pairwise_dist(a, b)
    kern_test_result = test_gaussian_kernel(a, b, 1)
    
    coeff_test_result = test_kernel_ridge_coeff()
    prediction_test_result = test_kernel_ridge_prediction()
    
    return(dist_test_result, kern_test_result)

In [19]:
def main(k=5):
    '''
    --------------------------
    Load data from source
    Input: Path to data
    Output: Data-frame
    -------------------------
    '''
    # Setup
    path = 'http://www0.cs.ucl.ac.uk/staff/M.Herbster/boston-filter/Boston-filtered.csv'
    
    # Set parameters for grid search
    start_g = 26
    stop_g = 40
    start_s = 7
    stop_s = 13
    
    # Get list of parameters to search over
    gamma_list, sigma_list = get_grid_search_vals(start_g, stop_g, start_s, stop_s)
    
    # Load data
    df = get_data(path)
    
    # Split into train and test: we are making 2/3 splits as required by default
    df_train, df_test = train_test_split(df)

    # Divide training data into k=5 folds for cross-validation as question asks
    df_folds = get_k_folds(df_train, k)
    
    # Convert data-frames into numpy arrays
    X_folds = [df_fold.iloc[:,:12].to_numpy() for df_fold in df_folds]
    Y_folds = [df_fold['MEDV'].to_numpy() for df_fold in df_folds]
    
    # Create a container to store results
    history = {'gamma': [],
               'sigma': [],
               'mse_val': [],
               'mse_train': [], 
               'pred_train': [],
               'pred_val': [],
               'alpha': []}
    
    counter = 0
    
    # This loop iterates over the grid
    for sigma in sigma_list:
        for gamma in gamma_list:
            
            # Record the parameter values
            history['sigma'].append(sigma)
            history['gamma'].append(gamma)
            
            # Print progress check
            counter += 1
            print(counter)
            
            # Make containers for train and validation MSE by fold
            mse_train_by_fold = []
            mse_val_by_fold = []
    
            # Now train for each fold
            for i in range(k):
        
                # Stack data matrices
                X_train_fold = np.vstack(X_folds[0:i] + X_folds[i+1:])
                Y_train_fold = np.hstack(Y_folds[0:i] + Y_folds[i+1:])
            
                # Get the ith fold and take it as the validation labels
                # The rest of the data are stacked and put in a container
                X_val_fold = X_folds[i]
                Y_val_fold = Y_folds[i]
        
                # Train the ridge regression
                alpha, y_pred_train, y_pred_val, mse_train, mse_val = train_kernel_ridge(X_train_fold, 
                                                                                         X_val_fold, Y_train_fold, 
                                                                                         Y_val_fold, sigma, gamma)
                
                # Store the result
                mse_train_by_fold.append(mse_train)
                mse_val_by_fold.append(mse_val)
                
            print(mse_train_by_fold)
            print(mse_val_by_fold)
            
            history['alpha'].append(alpha)
            history['pred_train'].append(y_pred_train)
            history['pred_val'].append(y_pred_val)
            history['mse_val'].append(np.mean(np.array(mse_val_by_fold)))
            history['mse_train'].append(np.mean(np.array(mse_train_by_fold)))
            
    best_params = get_best_params(history)
        
    return(history, best_params)

In [20]:
history, best_params = main()

1
[1.8535968822903233, 1.6303227795412492, 2.1388021699113597, 1.0349575087698328, 0.8099556923539831]
[80.26527998517186, 76.55638247771057, 92.95038176511945, 1108.279507406872, 56.23824914102332]
2
[1.5082135194441095, 1.384516618904326, 1.8590008757859133, 0.7992857761674441, 0.5655072295506672]
[107.30219881505333, 130.9475814878212, 115.33029722765559, 1272.4672768434932, 74.02767066101748]
3
[1.17582978425696, 1.1618064420368959, 1.5703609685754047, 0.6049350011306889, 0.3834483523761911]
[138.35052639741727, 216.7712003414733, 131.75282468494507, 1413.5364157231522, 97.17737946720888]
4
[0.8881335050060276, 0.9746376251407273, 1.2933476649069127, 0.4400568911462018, 0.2627526970701295]
[164.336980991176, 313.0083902776635, 137.5090773060237, 1628.7910090361806, 118.48116200489143]
5
[0.6566110726231943, 0.8186817226028613, 1.0445586122149886, 0.309752610206579, 0.18469846494336886]
[176.16743078187693, 406.6046325683571, 141.7368349320478, 1970.573705841049, 130.4089796558107]


[0.2323851758042905, 0.35858360857431476, 0.3865773557367063, 0.11444354060345586, 0.06372221369772398]
[824.1592457498721, 3226.674582822086, 836.7781910373568, 1177.0118196043468, 591.4179016712227]
46
[4.869781634146622, 4.247594519593817, 5.023669239363784, 4.119539714903537, 3.381023706583519]
[13.699528296743333, 18.751463204473637, 6.097533180396356, 17.3612574688257, 16.56792645712142]
47
[4.357982787131642, 3.7414016824134047, 4.51013020658611, 3.686056798490848, 2.9255873064423783]
[17.332388419236576, 23.482413969174708, 6.318284077495338, 17.09542816434599, 17.850155585424584]
48
[3.902337325171633, 3.3098819388000176, 4.0606305360379435, 3.286654903073754, 2.540212709832393]
[21.252928334644718, 30.883541819208308, 6.60340425282528, 18.290613977406807, 20.053742642982865]
49
[3.508747436194648, 2.9662797056025934, 3.685505819085952, 2.901848095225467, 2.2189920945669055]
[25.0933315235675, 40.99429497087382, 7.258079906043303, 21.878868689646197, 23.220449045760898]
50
[3.

[8.156196729585002, 6.8522210372524635, 7.849251760739153, 7.0521946164991896, 6.667719064893923]
[7.899481795417292, 15.148079182061375, 10.755518879483013, 14.810393198110308, 13.697368934067553]
93
[7.7601179798320485, 6.534216431871941, 7.521997796998973, 6.655074746942292, 6.232307321073564]
[7.6403881773331115, 15.174728245194357, 10.24173080861785, 15.52172820992391, 13.654206333481856]
94
[7.343076297851862, 6.225740416996529, 7.185736151097521, 6.261817638473163, 5.785054215532314]
[7.46241703985517, 15.074416282437426, 9.440983399775698, 16.231266448265618, 13.759092490161981]
95
[6.921377946521035, 5.915521673676832, 6.828451010908189, 5.878951354390073, 5.343327479777674]
[7.424665634245629, 14.870990719698998, 8.540505003547958, 17.005020061802735, 13.985661272063158]
96
[6.501628278259482, 5.583991627910336, 6.440024186853328, 5.5060689744681, 4.90600700873615]
[7.5195402174273624, 14.736263163779435, 7.7509233526887344, 17.848128367653313, 14.288923275231078]
97
[6.06682

[9.850362448638117, 8.666084395381652, 9.5165463223549, 9.207154327636562, 8.565973137573302]
[9.879408978933702, 15.47880534489861, 11.126228798438541, 12.791648587071741, 15.04544649001267]
141
[9.471228492315117, 8.221639009416377, 9.057512196886425, 8.800595644972812, 8.130954345447282]
[9.415484187207202, 15.401491901191749, 12.026517014590652, 12.52063000943956, 14.6216521862035]
142
[9.094777620722178, 7.791476858273235, 8.6573353019678, 8.388344791555356, 7.735151971282657]
[9.015817553093026, 15.508210517602093, 12.806110514920903, 12.503976134814232, 14.340616776453297]
143
[8.718808448587112, 7.387255978052849, 8.304273431213115, 7.942251112255116, 7.3511022739349885]
[8.774343048053533, 15.790031374074577, 13.283217593172184, 12.767017682470353, 14.112610437300145]
144
[8.341175073437547, 7.022043977779259, 7.983567357963398, 7.464436670556893, 6.954285897979941]
[8.64020574298722, 16.151649827873545, 13.295710993732351, 13.32947287546058, 13.919484040911147]
145
[7.9548156

[22.655836949564875, 18.971498120576435, 21.924713595367532, 20.59572213067273, 18.67370788146447]
[14.764455266696139, 31.959708726300892, 18.571854385078527, 28.3572847389918, 34.2811662219925]
184
[20.517116934569913, 17.443991343514277, 19.93538433154221, 18.876141883889254, 17.111630305162553]
[13.782533763345112, 29.66295123831591, 17.067124054619544, 25.851818526990296, 31.918120488591985]
185
[17.9299008950597, 15.598360000521534, 17.5453882791881, 16.71653190572915, 15.212635323421242]
[12.640518658325389, 26.44926454739693, 15.260441315234061, 22.637131091818027, 28.727347946038662]
186
[15.344288367997418, 13.675075943897813, 15.171180051462395, 14.446205387624198, 13.305557553097382]
[11.652089742955484, 22.88158207643663, 13.389462777007996, 19.33910140950157, 25.170845714766656]
187
[13.260782868884922, 12.010781088076518, 13.235953019939386, 12.529240590772309, 11.746994416946784]
[11.072977653475364, 19.792728473687635, 11.76133751183376, 16.750190122620264, 21.84908058

In [21]:
best_params

{'best_run': 94,
 'best_gamma': 9.313225746154785e-10,
 'best_sigma': 1024.0,
 'best_val_mse': 12.365368538271696,
 'best_run_alpha': array([[ 3.97717871e+09],
        [ 5.09907583e+08],
        [ 1.70113799e+09],
        [-2.68010209e+09],
        [-2.05014085e+09],
        [-3.06754678e+08],
        [ 6.40850906e+08],
        [-1.15250374e+09],
        [-6.57116770e+08],
        [ 1.47470740e+09],
        [-3.00884375e+09],
        [ 8.12362891e+08],
        [-5.84606586e+08],
        [ 3.47707953e+08],
        [-1.32133571e+09],
        [ 1.08610010e+09],
        [-9.25491212e+06],
        [ 9.74097309e+08],
        [-5.10541659e+09],
        [ 2.76124701e+07],
        [-1.39323296e+09],
        [-1.15107168e+09],
        [ 5.10747325e+08],
        [ 2.38210269e+08],
        [-9.77186075e+08],
        [ 2.12332555e+08],
        [-4.90818814e+08],
        [-7.76854407e+07],
        [ 2.48409480e+09],
        [-1.34445991e+09],
        [ 3.26842949e+09],
        [ 8.92456423e+08],
   